<a href="https://colab.research.google.com/github/spped2000/Thai-Handwriting-Recognition/blob/main/HandWritingRecog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Thai Letter Classification

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Loading and Preprocessing data
filelist = glob.glob("/content/drive/MyDrive/Colab Notebooks/DeepLearnin/FaceClassify/Project/set2*.jpg")
img = np.zeros(dtype= 'uint8', shape=(len(filelist), 256, 256, 3))
for i in range(len(filelist)):
  x = Image.open(filelist[i])
  #y = x.resize((256,256))
  #z = asarray(y)
  #img[i,:,:,:] = z[:,:,:1]

In [ ]:
# Define Thai letters array
# 01 = ก, 02 = ข, ...
#letters=u'กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮ'
classes = ['ก','ข', 'ฃ', 'ค', 'ฅ', 'ฆ','ง','จ','ฉ','ช','ซ','ฌ','ญ','ฎ','ฏ','ฐ','ฑ','ฒ','ณ','ด','ต','ถ','ท','ธ','น','บ','ป','ผ','ฝ','พ','ฟ','ภ','ม','ย','ร','ล','ว','ศ','ษ','ส','ห','ฬ','อ','ฮ']

#classes = ['01','02','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','43','44']

In [ ]:
# Selecting data
image_size = (28, 28)
batch_size = 32

# traning set
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Colab Notebooks/DeepLearnin/Project/set2",
    validation_split=0.1,
    class_names = classes,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
# validation set
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Colab Notebooks/DeepLearnin/Project/set2",
    validation_split=0.1,
    class_names = classes,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

In [ ]:
# Visulize data

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

In [ ]:
#data augmentation
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ])
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
# augmented training set
augmented_train_ds = train_ds.map(  lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
#Configure the dataset for performance
train_ds = train_ds.prefetch(buffer_size=batch_size)
val_ds = val_ds.prefetch(buffer_size=batch_size)

In [ ]:
#Build a model
num_classes = len(classes)
model = Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(28, 28, 3)),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),

  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),

  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.1), # Dropout 
  #layers.add(BatchNormalization()) # Batch normalization
  # normalizes the inputs heading into the next layer, ensuring that the network always creates activations with the same distribution that we desire
  layers.Dense(num_classes),
  layers.Activation('softmax')
])

In [ ]:
#Train the model
epochs=20

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
#model.save_weights(top_model_weights_path)
score = model.evaluate(val_ds, batch_size=batch_size, verbose=1)
print("score :", score[0])
print("accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("Loss: {}".format(eval_loss)) 


In [ ]:
model.summary()

In [ ]:
#Graphing our training and validation
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
#plt.save('/content/drive/MyDrive/Colab Notebooks/DeepLearnin/Project/result.jpg')

In [ ]:
img = keras.preprocessing.image.load_img(
    '/content/drive/MyDrive/Colab Notebooks/DeepLearnin/Project/testing_set/t1.jpg', target_size=(28, 28)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print("This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(classes[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
pickle_out = open("/content/drive/MyDrive/Colab Notebooks/DeepLearnin/Project/model_trained.p", "wb")
pickle.dump(model, pickle_out)
picke_out.close()